# Image Classification 

* Dataset: [Fashion MNIST](https://www.kaggle.com/zalando-research/fashionmnist)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Images
* **Images**: Matrix of numbers (pixels). Each pixel holds a number we will work with
* **Images with colors**: Multi channel. 3 values needed.
* **Greyscale images**: Single channel images. 1 value needed.
* Every image is 28x28 pixels.

# 4 values

* **List of images**: (1, 2, 3, 4)
* **Value 1**: Number of images in the list
* **Value 2,3**: Height and width of the image
* **Value 4**: Number of channels in the image

In [ ]:
# Get data
#data = 'mnist.csv'
data = 'https://raw.githubusercontent.com/helenabarmer/machine_learning_intro/master/Multiclass%20Classification/mnist.csv'
df = pd.read_csv(data)
df.head()

In [ ]:
# Shape of the data
df.shape

In [ ]:
# Features
X = df[df.columns[1:]]

# Labels
y = df['label']

In [ ]:
df.describe()

## Let us scale our data as we can see the variance of the mean.
Let's scale the pictures so they are at a value between 0 and 1.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fit_scaler = scaler.fit(X)
scaled_pixels = scaler.transform(X)

In [ ]:
# Helper function
def display_image(features, image_label):

    # Labels
    label_names = {
        0: 'T-shirt',
        1: 'Trouser',
        2: 'Pullover',
        3: 'Dress',
        4: 'Coat',
        5: 'Sandal',
        6: 'Shirt',
        7: 'Sneaker',
        8: 'Bag',
        9: 'Ankle boot'
    }
    
    print('This is a', label_names[image_label].lower())
    plt.imshow(features.reshape(28,28))

In [ ]:
# Test to display image
display_image(X.loc[5].values, y.loc[5])

## Split our data
We will go ahead and split our data into test and train sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

## Fit model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
predict = model.predict(X_test)
accuracy = accuracy_score(y_test, predict, normalize=True)
print(f'Accuracy of model: {accuracy*100}%')

## Neural networks
CNN with PyTorch

In [ ]:
import torch
import torchvision

cuda = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(cuda)

In [ ]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
print("X_train: ", X_train_tensor.shape)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float)
print("X_test: ", X_test_tensor.shape)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
print("y_train: ", y_train_tensor.shape)

y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
print("y_test: ", y_test_tensor.shape)

## Reshape images to 1, 28x28

In [ ]:
X_train_tensor = torch.Tensor(X_train_tensor.reshape((-1, 1, 28, 28)))

X_test_tensor = torch.Tensor(X_test_tensor.reshape((-1, 1, 28, 28)))

print("X_train: ", X_train_tensor.shape)
print("X_test: ", X_test_tensor.shape)

## Setting up our neural network

In [ ]:
# Input size, for grayscale 1 and for color 3
input_size = 1

# Hidden layers
hidden_1 = 1
hidden_2 = 32

# Output values, we have 10 different labels
print(f'Check number of labels: {len(df["label"].unique())}')
output = 10
      
nn_size = 5

## Setting up the model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class CNN_Model(nn.Module):
    
    def __init__(self):
        super(CNN_Model, self).__init__()
        
        # Layer 1
        self.layer_1 = nn.Sequential(
            nn.Conv2d(input_size, hidden_1, nn_size),
            nn.BatchNorm2d(hidden_1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # Layer 2
        self.layer_2 = nn.Sequential(
            nn.Conv2d(input_size, hidden_2, nn_size),
            nn.BatchNorm2d(hidden_2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # Final layer
        self.final_layer = nn.Linear(512, output)
        
    def forward(self, x):
        
        out = self.layer_1(x)
        print("Layer 1: ", out.shape)

        out = self.layer_2(out)
        print("Layer 2: ", out.shape)

        out = out.reshape(out.size(0), -1)

        out = self.final_layer(out)
        print("Final layer: ", out.shape)
        
        return out

In [ ]:
# Move model to cuda
model = CNN_Model()
model.to(cuda)

In [ ]:
# Move X and y to cuda
X_train_tensor = X_train_tensor.to(cuda)
X_test_tensor = X_test_tensor.to(cuda)

y_train_tensor = y_train_tensor.to(cuda)
y_test_tensor = y_test_tensor.to(cuda)

## Parameters

In [ ]:
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
epochs = 100
loss_list = list()

for epoch in range(1, epochs):
    
    # Calculate output and loss
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Zero out gradient
    optimizer.zero_grad()
    
    # Backward pas
    loss.backward()
    
    # Update model parameters
    optimizer.step()
    
    print(f'Epoch: {epoch} Loss {loss.item()}')
    loss_list.append(loss.item())

## Plot loss values

In [ ]:
x = (range(0, 99))

plt.figure(figsize=(12,8))
plt.plot(x, loss_list, 'go--', linewidth=2, markersize=12, color='purple')
plt.xlabel('Epoch', fontsize=22, color='white')
plt.ylabel('Loss', fontsize=22, color='white')
plt.gcf().set_facecolor("purple");

## Evaluate model

In [ ]:
model.eval()

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    
    y_test = y_test_tensor.cpu().numpy()
    predicted = predicted.cpu()
    
    accuracy_cnn = accuracy_score(predicted, y_test)
    
    print(f'Accuracy: {accuracy_score(predicted, y_test)}')

## Conclusions

In [ ]:
# Compared models
print(f'Accuracy of LogisticRegression: {accuracy*100}%')
print(f'Accuracy of CNN/PyTorch: {accuracy_cnn*100}%')

In [ ]:
Our neural network did better during the image classification.